In [1]:
from gurobipy import *


In [2]:
milp_model = Model("milp")

Restricted license - for non-production use only - expires 2027-11-29


In [3]:
# # Gurobi WLS license file
# # Your credentials are private and should not be shared or copied to public repositories.
# # Visit https://license.gurobi.com/manager/doc/overview for more information.
# WLSACCESSID=5567d935-0c99-4630-877a-2f0faa1e1d46
# WLSSECRET=306c2296-f7d9-4eec-aab4-8966fdf8d9ac
# LICENSEID=2725120


import gurobipy as gp

# <<< REPLACE WITH YOUR REAL VALUES (as strings) >>>
ACCESS_ID = "d06a70a9-a575-46f5-b3d3-df24ed75f420"   # string!
SECRET    = "4028d0de-05b9-4ab2-8917-4ee328c3a7b9"   # string!
LICENSEID = 2783276                                # string is safest

# Quick type check (optional)
assert isinstance(ACCESS_ID, str) and isinstance(SECRET, str) and isinstance(LICENSEID, (str, int))

env = gp.Env(empty=True)
env.setParam("WLSACCESSID", str(ACCESS_ID))
env.setParam("WLSSECRET",   str(SECRET))
env.setParam("LICENSEID",   LICENSEID)   # pass as string to be safe
# env.setParam("WLSSERVER", "https://license.gurobi.com")  # optional
env.start()

# smoke test
m = gp.Model("ping", env=env)
x = m.addVar(ub=1.0)
m.setObjective(x, gp.GRB.MAXIMIZE)
m.optimize()
print("Gurobi OK, x =", x.X)


Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2783276
Academic license 2783276 - for non-commercial use only - registered to ro___@gmail.com
Gurobi Optimizer version 13.0.1 build v13.0.1rc0 (mac64[arm] - Darwin 23.0.0 23A344)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Academic license 2783276 - for non-commercial use only - registered to ro___@gmail.com
Optimize a model with 0 rows, 1 columns and 0 nonzeros (Max)
Model fingerprint: 0x5939e866
Model has 1 linear objective coefficients
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]

Presolve removed 0 rows and 1 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01

In [7]:

import re
from typing import Dict, Tuple, Optional, Iterable
import gurobipy as gp

# ----------------- bit helpers -----------------
def rotl(bits, r: int):
    n = len(bits); r %= n
    return [bits[(i - r) % n] for i in range(n)]

def rotr(bits, r: int):
    n = len(bits); r %= n
    return [bits[(i + r) % n] for i in range(n)]

def bits_from_word(m: gp.Model, name: str, nbits: int):
    return [m.addVar(vtype=gp.GRB.BINARY, name=f"{name}[{i}]") for i in range(nbits)]

def add_word(m: gp.Model, x, y, name: str):
    """Mod-2^n add encoded exactly per bit: x + y + cin = z + 2*cout, cin=0."""
    n = len(x)
    z = [m.addVar(vtype=gp.GRB.BINARY, name=f"{name}_z[{i}]") for i in range(n)]
    c = [m.addVar(vtype=gp.GRB.BINARY, name=f"{name}_c[{i}]") for i in range(n+1)]
    m.addConstr(c[0] == 0, name=f"{name}_cin0")
    for i in range(n):
        m.addConstr(x[i] + y[i] + c[i] == z[i] + 2*c[i+1], name=f"{name}_fa[{i}]")
    return z, c[1:]  # carry-outs c[1]..c[n] (c[n] is overflow)

def xor2_word(m: gp.Model, a, b, name: str):
    """Bitwise XOR via linearization: a + b = z + 2*(a&b)."""
    n = len(a)
    z = [m.addVar(vtype=gp.GRB.BINARY, name=f"{name}_z[{i}]") for i in range(n)]
    t = [m.addVar(vtype=gp.GRB.BINARY, name=f"{name}_t[{i}]") for i in range(n)]
    for i in range(n):
        m.addConstr(t[i] <= a[i]); m.addConstr(t[i] <= b[i]); m.addConstr(t[i] >= a[i] + b[i] - 1)
        m.addConstr(a[i] + b[i] == z[i] + 2*t[i], name=f"{name}_xor[{i}]")
    return z

# --------------- MILP builder ----------------
def build_iosha_milp(
    rounds: int = 6,
    nbits: int = 64,
    lanes: int = 8,
    ROT_A: Optional[Iterable[int]] = None,
    ROT_B: Optional[Iterable[int]] = None,
    ROT_G: Optional[Iterable[int]] = None,
    iterative: bool = True,
    pins: Optional[Dict[str, int]] = None,
    mode: str = "feas_le",                 # 'min' | 'feas_ge' | 'feas_le'
    target_weight: int = 255,              # used by feas modes
    cube_t0_bits: int = 0,                 # fix low bits of round-0 parity word t
    cube_t0_value: int = 0,                # value for the fixed bits
    threads: int = 1,
    timelimit: Optional[int] = None,
    env: Optional[gp.Env] = None,
) -> Tuple[gp.Model, gp.LinExpr]:
    """
    Builds (but does NOT optimize) the MILP for IOSHA XOR-differentials.

    Returns: (model, W) where W is the linear expression for total carry weight
             (sum of carry-outs from all Mix-1 adds, overflow excluded).

    Pins expect keys like 'L0_0', 'R0_0' (side L/R, lane index, round index).
    """
    # IOSHA rotations (mod word size). lanes=8 = ARX lanes inside F.
    if ROT_A is None: ROT_A = [63,19,39,15,23,31,11,5]
    if ROT_B is None: ROT_B = [21, 3,31,33,45, 5,17,7]
    if ROT_G is None: ROT_G = [25,45,51, 5,63,47, 7,1]
    ROT_A = list(ROT_A); ROT_B = list(ROT_B); ROT_G = list(ROT_G)
    if len(ROT_A) < lanes: ROT_A = (ROT_A * ((lanes + len(ROT_A)-1)//len(ROT_A)))[:lanes]
    if len(ROT_B) < lanes: ROT_B = (ROT_B * ((lanes + len(ROT_B)-1)//len(ROT_B)))[:lanes]
    if len(ROT_G) < lanes: ROT_G = (ROT_G * ((lanes + len(ROT_G)-1)//len(ROT_G)))[:lanes]

    m = gp.Model("iosha_xordiff", env=env) if env else gp.Model("iosha_xordiff")
    if threads:   m.Params.Threads   = threads
    if timelimit: m.Params.TimeLimit = timelimit

    # # Helpful default knobs
    # m.Params.MIPFocus       = 2      # focus on proving (incl. infeasibility)
    # m.Params.Heuristics     = 0.05   # spend less time on heuristics
    # m.Params.Cuts           = 3      # more aggressive global cuts
    # m.Params.CutPasses      = 3
    # m.Params.ImpliedCuts    = 2
    # m.Params.LiftProjectCuts= 2
    # m.Params.FlowCoverCuts  = 2
    # # m.Params.GomoryCuts     = 2
    # m.Params.Presolve       = 2
    # m.Params.PrePasses      = 10
    # m.Params.PreSparsify    = 1
    # m.Params.Symmetry       = 2
    # # Long runs: tree spill to disk (prevents slowdown)
    # m.Params.NodefileStart  = 4      # GB
    # m.Params.NodefileDir    = "/tmp" # fast SSD path
    # # m.Params.NodefileCompress = 1

    # State words: L[j][i], R[j][i] as bit arrays
    L = [[bits_from_word(m, f"L{j}_{i}", nbits) for i in range(rounds+1)] for j in range(lanes)]
    R = [[bits_from_word(m, f"R{j}_{i}", nbits) for i in range(rounds+1)] for j in range(lanes)]

    weight_bits = []  # collects carry bits (excluding final overflow) from Mix-1 adds

    for i in range(rounds):
        # 1) L_{i+1} = R_i
        for j in range(lanes):
            for b in range(nbits):
                m.addConstr(L[j][i+1][b] == R[j][i][b], name=f"copy_L[{j},{i},{b}]")

        # 2) F(R_i): Mix-1 (ADD), Mix-2 (XOR with ROTR), RC dropped in diff model, t parity, output mix with ROTL(t)
        V1, V2, V3 = [[] for _ in range(lanes)], [[] for _ in range(lanes)], [[] for _ in range(lanes)]
        for j in range(lanes):
            j1, j2 = (j+1) % lanes, (j+2) % lanes
            # v1 = R[j] + ROTL(R[j1], ROT_A[j])
            z1, carries = add_word(m, R[j][i], rotl(R[j1][i], ROT_A[j] % nbits), f"add_{i}_{j}")
            V1[j] = z1
            if carries:
                weight_bits += carries[:-1]  # exclude overflow carry
            # v2 = v1 XOR ROTR(R[j2], ROT_B[j])
            V2[j] = xor2_word(m, V1[j], rotr(R[j2][i], ROT_B[j] % nbits), f"xor2_{i}_{j}")
            V3[j] = V2[j]  # RC addition is dropped (XOR-diff neutral under this conservative relaxation)

        # t = XOR over lanes of V3[j]
        t = V3[0]
        for j in range(1, lanes):
            t = xor2_word(m, t, V3[j], f"tacc_{i}_{j}")

        # cubing hook: fix low bits of t at round 0
        if i == 0 and cube_t0_bits > 0:
            for b in range(cube_t0_bits):
                bitval = (cube_t0_value >> b) & 1
                t[b].LB = bitval
                t[b].UB = bitval

        # fo[j] = v3[j] XOR ROTL(t, ROT_G[j])
        FO = [None] * lanes
        for j in range(lanes):
            FO[j] = xor2_word(m, V3[j], rotl(t, ROT_G[j] % nbits), f"fo_{i}_{j}")

        # 3) R_{i+1} = L_i XOR FO
        for j in range(lanes):
            R[j][i+1] = xor2_word(m, L[j][i], FO[j], f"Rnext_{i}_{j}")

    # Iterative trail: (L0,R0) == (Lr,Rr)
    if iterative:
        for j in range(lanes):
            for b in range(nbits):
                m.addConstr(L[j][0][b] == L[j][rounds][b], name=f"iter_L[{j},{b}]")
                m.addConstr(R[j][0][b] == R[j][rounds][b], name=f"iter_R[{j},{b}]")

    # Two-lane pinning (symmetry breaking)
    if pins:
        for key, val in pins.items():
            mobj = re.fullmatch(r'([LR])(\d+)_(\d+)', key)
            if not mobj:
                raise ValueError(f"Bad pin key: {key}")
            side, jj, rnd = mobj.group(1), int(mobj.group(2)), int(mobj.group(3))
            word = L[jj][rnd] if side == 'L' else R[jj][rnd]
            for b in range(nbits):
                bit = (val >> b) & 1
                word[b].LB = bit
                word[b].UB = bit

    # Objective / feasibility
    W = gp.quicksum(weight_bits)
    if mode == "min":
        m.setObjective(W, gp.GRB.MINIMIZE)
    elif mode == "feas_ge":
        m.addConstr(W >= target_weight, name="W_ge")
        m.setObjective(0, gp.GRB.MINIMIZE)
    elif mode == "feas_le":
        m.addConstr(W <= target_weight, name="W_le")
        m.setObjective(0, gp.GRB.MINIMIZE)
    else:
        raise ValueError("mode must be 'min' | 'feas_ge' | 'feas_le'")

    return m, W


In [8]:
nbits = 64
seed  = 1 << (nbits - 1)

m, W = build_iosha_milp(
    rounds=5,     # only for demonstration perpose 
    nbits=nbits,
    lanes=8,
    iterative=False,                       # <---
    pins={"L0_0": 0x0, "R0_0": seed},
    mode="feas_le",                       
    target_weight=255,
    env=env,
)
m.Params.MIPFocus   = 1
m.Params.Heuristics = 0.5
m.Params.Cuts       = 2
m.Params.Presolve   = 2
m.Params.Threads    = 8
# m.Params.TimeLimit  = 500

m.optimize()
print("Status:", m.Status, "SolCount:", m.SolCount)

if m.Status == gp.GRB.INFEASIBLE:
    print("Proved: min W >= 256 (no trail with W <= 255)")
elif m.Status == gp.GRB.OPTIMAL and m.SolCount > 0:
    print("There *is* a trail with W <= 255")
else:
    print("Unknown / time limit")


Set parameter Threads to value 1
Set parameter MIPFocus to value 1
Set parameter Heuristics to value 0.5
Set parameter Cuts to value 2
Set parameter Presolve to value 2
Set parameter Threads to value 8
Gurobi Optimizer version 13.0.1 build v13.0.1rc0 (mac64[arm] - Darwin 23.0.0 23A344)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Non-default parameters:
Heuristics  0.5
MIPFocus  1
Cuts  2
Presolve  2
Threads  8

Academic license 2783276 - for non-commercial use only - registered to ro___@gmail.com
Optimize a model with 44841 rows, 31144 columns and 129600 nonzeros (Min)
Model fingerprint: 0x7498b50e
Model has 0 linear objective coefficients
Variable types: 0 continuous, 31144 integer (31144 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+02]

Presolve removed 14963 rows and 20862 columns
Presolve time: 0.45s
Presolved: 